# ChatGPT API Tutorial
Adapted by Uiwon Hwang from the blog post (https://docs.kanaries.net/ko/topics/ChatGPT/how-to-use-chatgpt-api-python)

This tutorial is made for the Generative AI Challenge in Gangwon SW Festival. <br>
We encourage all participants of the challenge to refer to this tutorial.

# Install package

In [1]:
#!pip install openai==0.28
#!pip install python-dotenv


  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.6 kB 682.7 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 607.2 kB/s eta 0:00:00
  Using cached aiohappyeyeballs-2.3.4-py3-none-any.whl.metadata (5.6 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp39-cp39-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp39-cp39-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp39-cp39-win_amd64.whl.metadata (32 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
   ---------------------------------------- 0.0/376.2 kB ? eta -:--:--
   ----------- ---------------------------- 112.6/376.2 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------  368.6/376.2 kB 4.6 MB/s eta 0:00:01


# Import package

In [4]:
import openai
import os
import pandas as pd
import time
from dotenv import load_dotenv

load_dotenv()

# API key setting
You can get API key using the following instruction:
1. Visit https://platform.openai.com/api-keys
2. Click on the "+ Create new secret key" button
3. Click "create secret key"
4. Copy and save your API key


- Make sure to keep your API key secure
- You should pay for ChatGPT API by visiting https://platform.openai.com/settings/organization/billing/overview and clicking on "Add payment details" button

# ChatGPT response function

In [5]:
def get_completion(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.0,
    )
    return response.choices[0].message["content"]

# Query to API

In [6]:
prompt = "생성형 AI 챌린지 참가자들에게 인사해 줘, 매 문장 끝에 \n을 삽입해 줘" #"<YOUR QUERY HERE>"
response = get_completion(prompt)
print(response)

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.